In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv("/kaggle/input/water-potability/water_potability.csv")
data.head()                                 #READ  DATASET AND HEAD

In [ ]:
data.describe()

In [ ]:
data.drop_duplicates() #BY COMPARING ACTUAL DATASET AND AFTER DROP_DUPLICATES WE CAME TO KNOW THAT NO DUPLICATE VALUE IS PRESENT

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
for i in data.columns:
    missing_value=data[i].isnull().sum()
    check_value=len(data)/2
    if missing_value > check_value:
        print("The column "+str(i)+" with "+str(missing_value)+" is not valid for prediction,the missing values are higher then half of dataset")
    else:
        print("The column "+str(i)+" with "+str(missing_value)+" is valid for prediction,the missing values are lesser then half of dataset")

In [ ]:
data["ph"].fillna(data["ph"].mean(),inplace=True)
data["Sulfate"].fillna(data["Sulfate"].mean(),inplace=True)
data["Trihalomethanes"].fillna(data["Trihalomethanes"].mean(),inplace=True)

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns
sns.distplot(data["Potability"])
plt.title("ditribution of data(Potability) is expressed in graph")
plt.show()

In [ ]:
sns.relplot(data["ph"],data["Hardness"],hue=data["Potability"])
plt.title("ph vs Hardness")
plt.show()

In [ ]:
sns.relplot(data['ph'],data['Organic_carbon'],hue=data["Potability"],kind="line")
plt.show()

In [ ]:
sns.pairplot(data)
plt.show()

In [ ]:
sns.heatmap(data)
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix
from sklearn.model_selection import GridSearchCV

In [ ]:
x=data.drop("Potability",axis=1)
y=data["Potability"]
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.30,random_state=42)

In [ ]:
model={
       "LogisticRegression":LogisticRegression(),
       "SGDClassifier":SGDClassifier(),
       "DecisionTreeClassifier":DecisionTreeClassifier(),
       "RandomForestClassifier":RandomForestClassifier(),
       "GaussianNB":GaussianNB(),
       "SVC":SVC(),
       "KNeighborsClassifier":KNeighborsClassifier()
      }

In [ ]:
name_=[]
score_=[]
accuracy_=[]
f1_score_=[]
col=[]
for name,models in model.items():
    mod=models.fit(X_train,Y_train)
    pred=mod.predict(X_test)
    name_.append(name)
    score_.append(mod.score(X_test,pred))
    accuracy_.append(accuracy_score(Y_test,pred))
    f1_score_.append(f1_score(Y_test,pred))
    col.append(0)
data1=pd.DataFrame({"Name":name_,"Score":score_,"Accuracy":accuracy_,"F1_score":f1_score_,"HyperParameterTuning":col})

In [ ]:
model2={"LogisticRegression":{"model":LogisticRegression(),
                             "param":{"penalty":['none','l2'],
                                     }
                            },
        "SGDClassifier":{"model":SGDClassifier(),
                         "param":{"loss":['hinge', 'log', 'modified_huber','squared_hinge', 'perceptron'],
                                  "penalty":['l2',"none"]
                                 }
                        },
       "DecisionTreeClassifier":{"model":DecisionTreeClassifier(),
                                 "param":{"criterion":["gini", "entropy"],
                                 "max_depth":[1,4,8,10],
                                 "max_features":["auto", "sqrt", "log2"]
                                 }
                                },
       "KNeighborsClassifier":{"model":KNeighborsClassifier(),
                               "param":{"n_neighbors":[1,2,3,4,5],
                                         "weights":['uniform', 'distance']
                               }
                              },
       "RandomForestClassifier":{"model":RandomForestClassifier(),
                                 "param":{
                                          "n_estimators":[100,200],
                                          "criterion":["gini", "entropy"]
                                 }
                                },
       "GaussianNB":{"model":GaussianNB(),
                     "param":{
                         "var_smoothing":[1e-2,1e-3,1e-3]
                     }
                    },
       "SVC":{"model":SVC(),
              "param":{
                  "C":[1.0,1.4,1.7,1.9,2.0]
              }
             }
      }

In [ ]:
name_=[]
score_=[]
accuracy_=[]
f1_score_=[]
col=[]
for name,models in model2.items():
    r=GridSearchCV(models["model"],models["param"],cv=2).fit(X_train,Y_train)
    pred=r.predict(X_test)
    name_.append(name)
    score_.append(mod.score(X_test,pred))
    accuracy_.append(accuracy_score(Y_test,pred))
    f1_score_.append(f1_score(Y_test,pred))
    col.append(1)
data2=pd.DataFrame({"Name":name_,"Score":score_,"Accuracy":accuracy_,"F1_score":f1_score_,"HyperParameterTuning":col})

In [ ]:
stat_HPT=pd.pivot_table(pd.concat([data1,data2],axis=0),index=["Name"],columns=["HyperParameterTuning"])
stat_HPT

SO WE FIND KNN PERFORM BETTER 